# Trabalho Final
Neste trabalho, vamos buscar o reconhecimento dos dígitos da Linguagem de Sinais. Para tal, vamos nos utilizar de Modelos conhecidos de Deep Learning e também nos aventurar na criação de próprios.

------

## Imports

In [ ]:
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [1]:
import os
import numpy as np
from random import sample, seed
seed(42)
np.random.seed(42)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15,15) # Make the figures a bit bigger

# Keras imports
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, GlobalAveragePooling2D, Dense, Flatten
from keras.models import Model
from keras.utils import np_utils
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions

import inf619utils

Using TensorFlow backend.


# Dataset
O dataset é composto por 10 classes (dígitos de 0 a 9) com aproximadamente 205 imagens por classe. 
O conjunto foi dividido em 60% para treinamento, 15% para validação e 20% para teste.
As imagens estão divididas em blocos por classe.

** IMPORTANTE NÃO ALTERAR O NOME/LOCAL DAS IMAGENS** 

In [9]:
keras_model_file = 'keras_resnet3.h5'

datasetDir = "./Dataset"
datasetDirSplitted = "./DatasetSplitted"
nbClasses = 10
input_shape = (224,224,3)
input_size = (224,224)

#Definir tamanho do batch e número de épocas
batch_size = 12 #check RAM or GPU memory size
epochs = 30

train_files = {}
val_files = {}
test_files = {}

train_files, val_files, test_files = inf619utils.splitData(datasetDir, nbClasses)

In [3]:
#plot the images from imgList
def plotImagesFromBatch(imgList):
    for i in range(len(imgList)):
        plotImage(imgList[i])


# Se quiser visualizar algum bloco de imagens, descomentar as linhas abaixo
# inf619utils.plotImages(val_files)
# inf619utils.plotImages(train_files)
# inf619utils.plotImages(test_files)

## Definição do modelo 

In [4]:
model = ResNet50(input_shape=input_shape, classes=nbClasses, include_top=False)
model.summary()

c:\users\rafae\anaconda3\envs\tensorflow\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation


# Modificação do modelo 

In [5]:
# Escolher a camada que será o ponto de partida 
x = model.output

#print([layer.name for layer in squeezeNetModel.layers])
#print("\n\nFreeze layers up until ", squeezeNetModel.layers[-20].name)

for layer in model.layers:
    layer.trainable = True#        layer.trainable = False

x = Flatten()(x)
x = Dense(10, activation='softmax')(x)


# Não se esqueça de definir o nome modelo, onde baseSqueezeNetModel 
# é o modelo base da Squeeze que vc definiu ali em cima
model = Model(model.inputs, x, name='resnet_new_adam')
#model2 = Model(squeezeNetModel.inputs, x, name='squeezenet_new_adadelta')
#model3 = Model(squeezeNetModel.inputs, x, name='squeezenet_new_sgd')

# Treinamento do Modelo

In [17]:
from keras.optimizers import SGD, Adam
#Compile o modelo
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

import keras.callbacks as callbacks

tbCallBack = callbacks.TensorBoard(log_dir = "./logs_resnet2")
tbEarly = callbacks.EarlyStopping(monitor='val_acc',min_delta=0,patience=8,verbose=0, mode='auto')
tbEarly100 = callbacks.EarlyStopping(monitor='acc', min_delta=0.0001, patience=3, mode='max')
tbModelChk = callbacks.ModelCheckpoint('.modelresnet_weights.hdf5', save_best_only=True, monitor='val_acc', mode='max')


In [10]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        rotation_range=90.,
        height_shift_range=0.2,
        width_shift_range=0.2,
        brightness_range=(1, 5))

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        datasetDirSplitted + '\\train',
        target_size=input_size,
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        datasetDirSplitted + '\\val',
        target_size=input_size,
        batch_size=batch_size,
        class_mode='categorical')

Found 1242 images belonging to 10 classes.
Found 309 images belonging to 10 classes.


In [22]:
#Fit nos dados
#Fit nos dados
hist = model.fit_generator(train_generator, 
                    steps_per_epoch= int(1242/batch_size), 
                    epochs = epochs,
                    validation_data = validation_generator,  
                    validation_steps = int(309/batch_size),
                    callbacks=[tbCallBack, tbEarly, tbEarly100, tbModelChk])

Epoch 1/30
103/103 [==============================] - 267s 3s/step - loss: 0.1597 - acc: 0.9434 - val_loss: 0.1052 - val_acc: 0.9697
Epoch 2/30
103/103 [==============================] - 266s 3s/step - loss: 0.2242 - acc: 0.9272 - val_loss: 0.5425 - val_acc: 0.9057
Epoch 3/30
103/103 [==============================] - 266s 3s/step - loss: 0.2093 - acc: 0.9417 - val_loss: 0.2708 - val_acc: 0.9428
Epoch 4/30
103/103 [==============================] - 266s 3s/step - loss: 0.2153 - acc: 0.9328 - val_loss: 0.1387 - val_acc: 0.9663


In [23]:
model.save(keras_model_file)

In [24]:
del model

# Aplicação do Modelo no Conjunto de Testes

In [ ]:
# Load Best weights saved
from keras.models import load_model

model = load_model(keras_model_file)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        datasetDirSplitted + '\\test',
        target_size=input_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=None)

In [ ]:
#Criação do generator p/ o conjunto de teste

#Teste
metrics = model.evaluate_generator(test_generator, 
                                   steps=511)

print("Test Loss ResNet---> ", metrics[0])
print("Test Accuracy ResNet---> ", metrics[1])    #Test is balanced, so Acc is normalized

# Gerar matrix de confusão

In [ ]:
predictions = model.predict_generator(test_generator)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())   

In [ ]:
len(test_generator.classes)

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report) 

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(true_classes, predicted_classes)

In [ ]:
from inf619utils import plot_confusion

plot_confusion(true_classes, predicted_classes, "ResNet")

# Teste do modelo

In [ ]:
from keras.preprocessing import image

img_paths = ['./Dataset/0/IMG_1118.JPG',
             './Dataset/1/IMG_1291.JPG',
             './Dataset/2/IMG_1272.JPG',
             './Dataset/3/IMG_4320.JPG',
             './Dataset/4/IMG_4729.JPG',
             './Dataset/5/IMG_4418.JPG',
             './Dataset/6/IMG_4430.JPG',
             './Dataset/7/IMG_4166.JPG',
             './Dataset/8/IMG_4274.JPG',
             './Dataset/9/IMG_4422.JPG'
            ]
             

for img_path in img_paths:  
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.true_divide(x, 255)
    x = np.expand_dims(x, axis=0)
    #x = preprocess_input(x)

    plt.imshow(img)
    plt.show()
    
    preds = model.predict(x)
    # decode the results into a list of tuples (class, description, probability)
    # (one such list for each sample in the batch)
    print('Predicted:', np.around(preds, decimals=3))
    print('Class:', preds.argmax(axis=-1))

# Salvar modelo TensorflowJS (Linux)

In [ ]:
#import tensorflowjs as tfjs
#tfjs.converters.save_keras_model(model, tfjs_target_dir)

# Salvar modelo TensorflowLite (Linux)

In [ ]:
#from tensorflow.contrib import lite

#converter = lite.TocoConverter.from_keras_model_file(keras_model_file)
#tflite_model = converter.convert()
#open("resnet.tflite", "wb").write(tflite_model)